<a href="https://colab.research.google.com/github/Liam-Nova/BOT-TELEGRAM/blob/main/Copia_de_BOT_TELEGRAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [ ]:
!pip install torch
!pip install transformers
!pip install openai
!pip install timm

import requests
from transformers import pipeline
from openai import OpenAI
from urllib.parse import urlparse
import openai

import re
from bs4 import BeautifulSoup

# https://www.semana.com/politica/,https://www.noticiascaracol.com/politica/

In [ ]:
# Inicializar el pipeline de Hugging Face para resumen
pipe = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


# Funciones

## Conectar a telegram, descarga de mensajes a colab y mensaje de bienvenida




In [ ]:

# Función para enviar mensajes
def send_messages(chat_id, text):
    url = f"{URL_BASE}sendMessage"
    params = {"chat_id": chat_id, "text": text}
    response = requests.get(url, params=params)
    return response

# Función para obtener actualizaciones del bot
def get_updates(offset=None):
    url = f"{URL_BASE}getUpdates"
    params = {"timeout": 100, "offset": offset}
    response = requests.get(url, params=params)
    return response.json()["result"]

# Función para enviar el mensaje de bienvenida
def send_welcome_message(chat_id):
    welcome_message = (
        "¡Hola! 👋 Bienvenido al Bot de Resumenes de Noticias 📰\n\n"
        "📌 **¿Qué puede hacer este bot?**\n"
        "Este bot te permite hacer resúmenes de artículos de noticias usando dos opciones:\n"
        "1. **Resumen con Hugging Face (BART)**: Utiliza el modelo BART de Hugging Face para generar resúmenes de las noticias.\n"
        "2. **Resumen con OpenAI (GPT)**: Usa OpenAI para generar resúmenes de manera más interactiva y cómica.\n\n"
        "📝 **¿Cómo funciona?**\n"
        "Envía las URLs de las noticias separadas por comas y luego selecciona una de las opciones para obtener un resumen.\n"
        "¡Vamos a comenzar! 🚀"
    )
    send_messages(chat_id, welcome_message)




 ## Limpian el contenido de las páginas

In [ ]:
def get_text_html(url): # extrae el texto de una página web
    try:
        response = requests.get(url, timeout=10)  # Agregar timeout para evitar bloqueos
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        return f"Error al obtener la URL: {e}"

# Función para limpiar texto
def text_clean(text):
    text = re.sub(r'@[A-Za-z0-9_]+', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'#', ' ', text)
    text = re.sub(r"[^A-Za-zÁÉÍÓÚáéíóúÜüÑñ0-9,\.\s'¿?¡!:\-]", ' ', text)
    text = re.sub(r'(?<=[a-záéíóúüñ])([A-ZÁÉÍÓÚÜÑ])', r' \1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Función para limpiar noticias
def clean_news_text(text):
    stop_phrases = [
        r'publicidad', r'suscribirse', r'iniciar sesión', r'síguenos.*', r'últimas noticias',
        r'tendencias', r'activar', r'notificaciones', r'facebook|twitter|instagram|youtube'
    ]
    for phrase in stop_phrases:
        text = re.sub(phrase, '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    content = [f"'{line.strip()}'" for line in re.split(r'[.?!]', text) if len(line.strip()) > 50]
    return content

## Validacion y primer tratamiento de las urls que es hacer el diccionario

In [ ]:
# Función para verificar si una URL es válida
def is_valid_url(url):
    parsed_url = urlparse(url)
    return bool(parsed_url.scheme) and bool(parsed_url.netloc)

# Generador de clave única para URLs en el diccionario
def generate_key_from_url(url):
    # Eliminar prefijos http:// o https://
    url = re.sub(r"https?://", "", url)
    # Eliminar "www.", ".com", ".org", ".net", y otros TLDs comunes
    url = re.sub(r"(www\.|\.com|\.org|\.net|\.co|\.es|\.gov|\.edu)", "", url)
    # Reemplazar cualquier "/" por "_"
    url = url.replace("/", "_").strip()
    return url

# librerias para cara usuario
user_libraries = {}

# Función para procesar las URLs y generar el diccionario
def process_urls(urls, chat_id):
    library = {}
    for url in urls:
        url = url.strip()
        if not is_valid_url(url):
            send_messages(chat_id, f"❌ La URL '{url}' no es válida. Por favor, ingresa una URL válida.")
            continue

        try:
            key = generate_key_from_url(url)  # Generador de clave única para URLs en el diccionario
            texto_crudo = get_text_html(url)  # Extrae el texto de una página web
            texto_limpio = text_clean(texto_crudo)  # Función para limpiar texto
            texto_depurado = clean_news_text(texto_limpio)  # Función para limpiar noticias
            library[key] = texto_depurado
        except Exception as e:
            send_messages(chat_id, f"Error procesando la URL {url}: {e}")

    if library:
        user_libraries[chat_id] = library  # Almacenar `library` en el diccionario de `user_libraries`

        for key, value in library.items():
            response = f"Clave: {key}\nContenido:\n" + "\n".join(value[:5])  # Limitar a 5 líneas
            send_messages(chat_id, response)

## Generacio de resumenes a partir de la opción escogida

In [ ]:
# Función para generar el resumen con Hugging Face
def generate_summary_huggingface(text):
    try:
        summary = pipe(text)
        return summary[0]['summary_text']
    except Exception as e:
        return f"Error al generar el resumen con Hugging Face: {e}"


# Función para generar el resumen con OpenAI
def generate_summary_openai(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "Eres un experto en noticias de actualidad."},
                  {"role": "user", "content": prompt}],
        max_tokens=150,
        temperature=0.1
    )
    return response["choices"][0]["message"]["content"]
     #return response.choices[0].message.content


# Función para elegir el tipo de resumen
def ask_for_summary_option(chat_id):
    send_messages(chat_id, "Por favor, elige el tipo de resumen:\n"
                            "1. **Resumen con Hugging Face (BART)**\n"
                            "2. **Resumen con OpenAI (GPT)**\n"
                            "Escribe 1 o 2 para elegir una opción.")

# Función para procesar el resumen basado en la opción seleccionada
def process_summary_choice(choice, chat_id):
    if chat_id not in user_libraries:
        send_messages(chat_id, "❌ No hay contenido para resumir. Asegúrate de haber procesado URLs primero.")
        return {}

    library = user_libraries[chat_id]  # Obtener el `library` para el usuario
    summary_library = {}  # Nuevo diccionario para almacenar los resúmenes

    for key, texto in library.items():
        if not texto:
            send_messages(chat_id, f"⚠️ No hay contenido para resumir en '{key}'.")
            continue

        if choice == "1":
            summary = generate_summary_huggingface(texto)
        elif choice == "2":
            summary = generate_summary_openai(texto)
        else:
            send_messages(chat_id, "❌ Opción no válida. Por favor, selecciona 1 o 2.")
            return {}

        summary_library[key] = summary  # Guardar el resumen en el nuevo diccionario
        send_messages(chat_id, f"🔹 Resumen para '{key}':\n{summary}")

    return summary_library  # Retornar el nuevo diccionario con los resúmenes

## Ruta para interactuar con el bot

In [ ]:
def main():
    print("Bot iniciado...")
    offset = None
    while True:
        try:
            updates = get_updates(offset)
            for update in updates:
                offset = update["update_id"] + 1
                if "message" in update:
                    chat_id = update["message"]["chat"]["id"]
                    text = update["message"].get("text", "")

                    # Verificar si el mensaje contiene "/start"
                    if text.startswith("/start"):
                        send_welcome_message(chat_id)  # Enviar el mensaje de bienvenida cuando el usuario envíe /start
                    elif text.startswith("http") or text.startswith("www"):
                        urls = text.split(",")  # Dividir las URLs que el usuario ingrese
                        process_urls(urls, chat_id)  # Procesar las URLs enviadas por el usuario
                        ask_for_summary_option(chat_id)  # Preguntar por el tipo de resumen
                    elif text == "1" or text == "2":
                        process_summary_choice(text, chat_id)  # Pasar el `chat_id` a la función para resumir
                    else:
                        # Esta es una opción extra si quieres darle al usuario algo más cuando escriba texto diferente
                        send_messages(chat_id, "🌐 Envíame una URL para procesar o usa /start para comenzar.")
        except Exception as e:
            print(f"Error en el loop principal: {e}")

# Datos iniciales

In [ ]:
# Código de interacción con telegram

TOKEN = 'My_token_Telegram'  # Reemplaza con tu token real de Telegram
URL_BASE = f"https://api.telegram.org/bot{TOKEN}/"

# Código de interacción con OpenAI
client = openai.OpenAI(api_key="My_Token_OpenAI")  # Reemplaza con tu clave de API de OpenAI


# Inicio del bot

In [ ]:
if __name__ == "__main__":
    main()


Bot iniciado...
